<a href="https://colab.research.google.com/github/Mario-RJunior/web-scraping-ML/blob/master/scraping_Mercado_Livre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
import re
import numpy as np

In [2]:
re_precos = '<span class="price__fraction">(.*)</span>'
re_quartos = r'\| (.*) quarto'
re_areas = '> (.*) m² '

In [3]:
zonas = ['norte', 'sul', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193']
dados = {'zona': [],
         'quartos': [],
         'area': [],
         'preco': []}

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{zona}{pagina}'

    c = requests.get(url).content
    soup = BeautifulSoup(c, 'html.parser')

    preco = soup.find_all(name='span', attrs={'class': 'price__fraction'})
    atributos = soup.find_all(name='div', attrs={'class': 'item__attrs'})

    for p in preco:
      valor = re.findall(re_precos, str(p))[0]
      dados['preco'].append(valor)
    
    for atributo in atributos:
      n_quartos = re.findall(re_quartos, str(atributo))
      area = re.findall(re_areas, str(atributo))

      if n_quartos == []:
        n_quartos = np.nan
      else:
        n_quartos = n_quartos[0]
      dados['quartos'].append(n_quartos)

      if area == []:
        area = np.nan
      else:
        area = area[0]      
      dados['area'].append(area)

      dados['zona'].append(zona)
    sleep(5)

In [4]:
df = pd.DataFrame(dados)

In [5]:
df

,zona,quartos,area,preco
0,norte,1,70,600
1,norte,2,70,850
2,norte,1,40,850
3,norte,1,50,950
4,norte,5,150,2.000
...,...,...,...,...
955,oeste,2,125,1.200
956,oeste,1,32,730
957,oeste,3,123,2.500
958,oeste,1,50,950
